In [1]:
import json
import pandas as pd
from pathlib import Path
from jsonschema import validate, ValidationError
from datasets import Dataset

/home/fishmon/.conda/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the schema for the classification dictionary (outside the function is cleaner)
classification_schema = {
    "type": "object",
    "properties": {
        "answer": {"type": "integer", "minimum": 0, "maximum": 3},
        "classification": {"type": "string"},
        "reason": {"type": "string"},
        "confidence": {"type": "integer", "minimum": 0, "maximum": 100}
    },
    "required": ["answer", "classification", "reason", "confidence"]
}

def to_sharegpt_with_thought(system, input_suffix, dataset) -> Dataset:
    """
    Convert website classification dataset to ShareGPT format including reasoning ('thought'),
    with JSON validation and enhanced error handling, specifically checking for unhashable types.
    Returns a Hugging Face Dataset object.
    
    Args:
        system (str): System prompt
        input_suffix (str): Suffix to append to the human message (can be empty if not needed)
        dataset (pd.DataFrame): Input DataFrame with columns:
            ['Domain', 'Content', 'Label', 'classification', 'reason', 'confidence', 'thought']
            
    Returns:
        datasets.Dataset: Hugging Face Dataset with a 'conversations' column, 
                          where each row contains a list representing one conversation.
                          Returns an empty Dataset if input dataset is empty or all rows fail.
    """
    if not isinstance(dataset, pd.DataFrame) or dataset.empty:
        print("Input is not a valid or non-empty DataFrame. Returning empty Dataset.")
        # Return an empty Dataset with the expected structure
        return Dataset.from_dict({"conversations": []}) 
        
    # This list will temporarily hold the conversation lists
    conversation_data_list = [] 
    error_count = 0
    processed_count = 0

    human_template = f"{input_suffix}\nDomain: {{domain}}, Content: \"{{content}}\""
    if not input_suffix:
         human_template = f"Domain: {{domain}}, Content: \"{{content}}\""

    print(f"Starting conversion for {len(dataset)} rows...")

    for idx, row in dataset.iterrows():
        try:
            # --- Data Extraction and Basic Type Check ---
            domain = row.get('Domain', 'N/A') 
            content = str(row['Content']) if pd.notna(row['Content']) else "" 
            thought_text = str(row['thought']).strip() if pd.notna(row['thought']) else "No thought provided."
            
            label = row['Label']
            classification = row['classification']
            reason = row['reason']
            confidence = row['confidence']

            if pd.isna(label) or pd.isna(classification) or pd.isna(reason) or pd.isna(confidence):
                 raise ValueError("One or more required classification fields are NaN")

            if isinstance(label, (list, dict)) or \
               isinstance(classification, (list, dict)) or \
               isinstance(reason, (list, dict)) or \
               isinstance(confidence, (list, dict)):
                 raise TypeError("One or more classification fields contain unhashable list/dict types")

            human_value = human_template.format(domain=domain, content=content)
            
            # --- Prepare and Validate Classification Dictionary ---
            classification_dict = {
                "answer": int(label), 
                "classification": str(classification),
                "reason": str(reason), 
                "confidence": int(confidence)
            }
            validate(instance=classification_dict, schema=classification_schema) 
            
            # --- Serialize and Format Output ---
            final_json_str = json.dumps(classification_dict, ensure_ascii=False, indent=2) 
            gpt_value = f"<think>\n{thought_text}\n</think>\n```json\n{final_json_str}\n```"
            
            # This is the list for a single conversation
            conversation = [
                {"from": "system", "value": system},
                {"from": "human", "value": human_value},
                {"from": "gpt", "value": gpt_value} 
            ]
            
            # Append the conversation list to our temporary list
            conversation_data_list.append(conversation) 
            processed_count += 1

        # --- Error Handling ---
        except (ValidationError, ValueError, TypeError) as e: 
            error_count += 1
            # Avoid printing excessive errors if many occur
            if error_count < 20 or error_count % 100 == 0: 
                 print(f"Error processing row {idx} (Domain: {domain}): {type(e).__name__} - {str(e)}")
            continue 
        except Exception as e: 
             error_count += 1
             if error_count < 20 or error_count % 100 == 0:
                 print(f"UNEXPECTED Error processing row {idx} (Domain: {domain}): {type(e).__name__} - {str(e)}")
             continue

    print(f"\nConversion finished.")
    print(f"Successfully processed: {processed_count} rows")
    print(f"Errors encountered: {error_count} rows")

    # --- Convert the list of conversation lists to a Hugging Face Dataset ---
    if conversation_data_list:
        # Create the dictionary format expected by from_dict
        hf_dataset_dict = {"conversations": conversation_data_list} 
        # Create and return the Dataset object
        return Dataset.from_dict(hf_dataset_dict)
    else:
        print("No valid data processed. Returning empty Dataset.")
        # Return an empty Dataset with the expected structure
        return Dataset.from_dict({"conversations": []})

In [3]:
df = pd.read_csv('./dataset/harmful.csv')
with open('./prompt/labelling_promptv4.txt', 'r', encoding='utf-8') as f:
    system_prompt = f.read()

# Convert to ShareGPT format with Unicode preservation
dataset = to_sharegpt_with_thought(
    system=system_prompt,
    input_suffix="Classify the given URL as 0 (benign), 1 (gambling), 2 (pornography), or 3 (harmful). Output MUST be JSON.\n",
    dataset=df
)

# Now you can check the type and print the Dataset info
print("\nOutput Type:", type(dataset))
print("Dataset Info:")
print(dataset)

Starting conversion for 776 rows...

Conversion finished.
Successfully processed: 776 rows
Errors encountered: 0 rows

Output Type: <class 'datasets.arrow_dataset.Dataset'>
Dataset Info:
Dataset({
    features: ['conversations'],
    num_rows: 776
})


In [ ]:
print(f"Saving {len(dataset)} conversations to JSONL...")
# Save dataset in JSONL format
try:
    # with open('./dataset/harmful_sharegpt_thought2.jsonl', 'w', encoding='utf-8') as f:
    #     # Iterate directly through the list of conversation lists
    #     for conversation_list in dataset: 
    #         f.write(json.dumps(conversation_list, ensure_ascii=False) + '\n')
    with open('./dataset/harmful_sharegpt_thought2.jsonl', 'w', encoding='utf-8') as f:
        for item in dataset:
            f.write(json.dumps(item['conversations'], ensure_ascii=False) + '\n')
    print("Successfully saved data to harmful_sharegpt_thought3.jsonl")
except Exception as e:
     print(f"Error saving JSONL file: {e}")

In [4]:
dataset['conversations'][0]

[{'from': 'system',
  'value': 'You are an Expert Website Classifier tasked with categorizing websites (using provided Domain & Content) into distinct categories: **0 - Benign**, **1 - Gambling**, **2 - Pornography**, or **3 - Harmful**. Output strict JSON including classification, reason, and a point-based confidence score (0-100). This aids a sophisticated website prediction system for digital safety.\n\n### **Categories & Definitions:**\n\n-   **0 - Benign:** General info, news, entertainment, services, e-commerce, educational sites, blogs, informational pages, and general entertainment sites. No gambling/porn themes.\n    -   **Includes:** Educational discussion of sensitive topics (drugs, gambling, adult, illegal), suggestive content (dating, lingerie) *without* explicit material, drugs for educational or news purposes (e.g., addiction recovery, medical cannabis research).\n\n-   **1 - Gambling:** Promotes/facilitates betting, casino, poker, lottery, wagering. Includes online bett

In [5]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.5.1 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.10 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


Unsloth: Standardizing formats (num_proc=16): 100%|██████████| 776/776 [00:01<00:00, 521.86 examples/s]


In [6]:
dataset['conversations'][0]

[{'content': 'You are an Expert Website Classifier tasked with categorizing websites (using provided Domain & Content) into distinct categories: **0 - Benign**, **1 - Gambling**, **2 - Pornography**, or **3 - Harmful**. Output strict JSON including classification, reason, and a point-based confidence score (0-100). This aids a sophisticated website prediction system for digital safety.\n\n### **Categories & Definitions:**\n\n-   **0 - Benign:** General info, news, entertainment, services, e-commerce, educational sites, blogs, informational pages, and general entertainment sites. No gambling/porn themes.\n    -   **Includes:** Educational discussion of sensitive topics (drugs, gambling, adult, illegal), suggestive content (dating, lingerie) *without* explicit material, drugs for educational or news purposes (e.g., addiction recovery, medical cannabis research).\n\n-   **1 - Gambling:** Promotes/facilitates betting, casino, poker, lottery, wagering. Includes online betting platforms, cas

In [7]:
print("\nOutput Type:", type(dataset))
print("Dataset Info:")
print(dataset)


Output Type: <class 'datasets.arrow_dataset.Dataset'>
Dataset Info:
Dataset({
    features: ['conversations'],
    num_rows: 776
})
